# Problem Set 2 - Part 2: Inference

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the following instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

In this part you will work on a problem from the FraCaS test suite. The goal of this problem set is to extend the grammar for Cooper storage from the previous part, apply the sentences to a theorem prover and a model builder, analyse their results and conclude about the inference. Since every sentence may be translated to several formulae there will also be several inference steps and thus several results.



### Pre-requisite knowledge

From `problem-set-1`:
- First order logic
- Lambda calculus
- Feature unification context free grammar

From `problem-set-2-part1.ipynb`:
- Cooper storage



In [1]:
# This task requires NLTK and Jupyter Notebook (IPython package).
import nltk
from nltk.grammar import FeatureGrammar
from nltk.sem import cooper_storage as cs

from utils import display_latex, display_translation, display_tree, display, Markdown
from utils2 import sem_parser, evaluate_sentences, syntax, syntax_notv

read_expr = nltk.sem.Expression.fromstring


### 1.1 Extending the grammar [6 marks]

Extend the grammar below so that it covers the sentences of the inference problem based on FraCaS problem 057.

Several solutions are possible, also some that would not result in entailment. Our lexical knowledge about words leads us how to formalise them in first order logic. Therefore, make sure that you handle the PP attachment in a away that you will be able to show entailments between examples.

In [2]:
sentences = [
    'a Portuguese delegate published the result on climate',
    'a Portuguese delegate published a result',
    'a Portuguese delegate published the result',
    'a Portuguese published a result',
]

In [3]:
fcfg_storage_base = r"""
% start S

S[SEM=[CORE=<?subj(?vp)>, STORE=(?b1+?b2)]] -> NP[NUM=?n,SEM=[CORE=?subj, STORE=?b1]] VP[NUM=?n,SEM=[CORE=?vp, STORE=?b2]]

Nom[NUM=?n,SEM=?s] -> N[NUM=?n,SEM=?s]

VP[NUM=?n,SEM=?s] -> IV[NUM=?n,SEM=?s]
VP[NUM=?n,SEM=[CORE=<?v(?obj)>, STORE=(?b1+?b2)]] -> TV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]]
VP[NUM=?n,SEM=[CORE=<?v(?pp)(?obj)>, STORE=(?b1+?b2+?b3)]] -> DTV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]] PP[+TO,SEM=[CORE=?pp, STORE=?b3]]

PP[+TO, SEM=[CORE=?np, STORE=?b1]] -> P[+TO] NP[SEM=[CORE=?np, STORE=?b1]]
"""

fcfg_storage_lexicon = r"""
PropN[NUM=sg,SEM=[CORE=<\P.P(angus)>, STORE=(/)]] -> 'Angus'
PropN[NUM=sg,SEM=[CORE=<\P.P(cyril)>, STORE=(/)]] -> 'Cyril'
PropN[NUM=sg,SEM=[CORE=<\P.P(irene)>, STORE=(/)]] -> 'Irene'

Det[NUM=sg,SEM=[CORE=<\P Q.all x.(P(x) -> Q(x))>, STORE=(/)]] -> 'every'
Det[NUM=sg,SEM=[CORE=<\P Q.exists x.(P(x) & Q(x))>, STORE=(/)]] -> 'a'

N[NUM=sg,SEM=[CORE=<\x.library(x)>, STORE=(/)]] -> 'library'
N[NUM=sg,SEM=[CORE=<\x.girl(x)>, STORE=(/)]] -> 'girl'
N[NUM=sg,SEM=[CORE=<\x.boy(x)>, STORE=(/)]] -> 'boy'
N[NUM=sg,SEM=[CORE=<\x.book(x)>, STORE=(/)]] -> 'book'

IV[NUM=sg,SEM=[CORE=<\x.smile(x)>, STORE=(/)],TNS=pres] -> 'smiles' 

TV[NUM=sg,SEM=[CORE=<\X x.X(\y.read(x,y))>, STORE=(/)],TNS=pres] -> 'reads'

DTV[NUM=sg,SEM=[CORE=<\Y X x.X(\z.Y(\y.give(x,y,z)))>, STORE=(/)],TNS=pres] -> 'gives'

P[+to] -> 'to'
""" 

fcfg_storage_np = r"""
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?np, @x)>+?b1)]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=?b1]]
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?det(?nom), @x)>+?b1+?b2)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]
"""

The following code resolves the readings from the Cooper storage. We collect the readings in a dictionary where the key is a sentence string and the value is a list of readings.

In [4]:
# your answers
fcfg_storage_answers_1 = r"""
### Replace X with their proper representations
N[NUM=sg,SEM=[CORE=<\x.portugese(x)>, STORE=(/)]] -> 'Portuguese'
N[NUM=sg,SEM=[CORE=<\x.delegate(x)>, STORE=(/)]] -> 'delegate'
TV[NUM=sg,SEM=[CORE=<\X x.X(\y.publish(x,y))>, STORE=(/)],TNS=pret] -> 'published'
Det[SEM=[CORE=<\Q P.exists x.(Q(x) & P(x) & all y. (Q(y) implies (y=x)))>, STORE=(/)]] -> 'the'
N[NUM=sg,SEM=[CORE=<\x.result(x)>, STORE=(/)]] -> 'result'
P[+ON] -> 'on' 
N[NUM=sg,SEM=[CORE=<\x.climate(x)>, STORE=(/)]] -> 'climate'
## Extend the other missig rules too
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?det(?nom1)(?nom2), @x)>+?b1+?b2+?b3)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom1, STORE=?b2]]  Nom[NUM=?n,SEM=[CORE=?nom2, STORE=?b3]]
#NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?det(?nom1(?nom2)), @x)>+?b1+?b2+?b3)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom1, STORE=?b2]]  Nom[NUM=?n,SEM=[CORE=?nom2, STORE=?b3]]
#PP[+ON, SEM=[CORE=?np, STORE=?b1]] -> P[+ON] NP[SEM=[CORE=?np, STORE=?b1]]

#PP[+ON, SEM=[CORE=?np, STORE=?b1]] -> P[+ON] N[NUM=?np, STORE=?b1]
PP[SEM=(?p + ?np)] -> P[+ON, SEM=?p] NP[SEM=?np]
"""

# this is going to add new rules to the syntax:
fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon + fcfg_storage_answers_1
new_syntax = FeatureGrammar.fromstring(fcfg_storage)

In [5]:
sentence_readings = sem_parser(sentences, new_syntax, verbose=False, is_cs=True)
#    'a Portuguese delegate published the result on climate',
#    'a Portuguese delegate published a result',
#    'a Portuguese delegate published the result',
#    'a Portuguese published a result',
# print all readings
for i, (sent, semreps) in enumerate(sentence_readings.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

1. a Portuguese delegate published the result on climate
2. a Portuguese delegate published a result


"1": $\exists\ x.(result(x)\ \land\ \exists\ z_{1}.(portugese(z_{1})\ \land\ delegate(z_{1}))(\lambda\ @x2.publish(@x2,x)))$

"2": $\exists\ x.(portugese(x)\ \land\ delegate(x))(\lambda\ @x2.\exists\ x.(result(x)\ \land\ publish(@x2,x)))$

3. a Portuguese delegate published the result


"1": $\exists\ x.(result(x)\ \land\ \exists\ z_{2}.(portugese(z_{2})\ \land\ delegate(z_{2}))(\lambda\ @x2.publish(@x2,x))\ \land\ \forall\ y.(result(y)\ \to\ (y\ =\ x)))$

"2": $\exists\ x.(portugese(x)\ \land\ delegate(x))(\lambda\ @x2.\exists\ x.(result(x)\ \land\ publish(@x2,x)\ \land\ \forall\ y.(result(y)\ \to\ (y\ =\ x))))$

4. a Portuguese published a result


"1": $\exists\ x.(result(x)\ \land\ \exists\ z_{3}.(portugese(z_{3})\ \land\ publish(z_{3},x)))$

"2": $\exists\ x.(portugese(x)\ \land\ \exists\ z_{4}.(result(z_{4})\ \land\ publish(x,z_{4})))$

In [6]:
# make sure understanding this data structure:
sentence_readings

{'a Portuguese delegate published the result on climate': [],
 'a Portuguese delegate published a result': [<ExistsExpression exists x.(result(x) & exists z1.(portugese(z1) & delegate(z1))(\@x2.publish(@x2,x)))>,
  <ApplicationExpression exists x.(portugese(x) & delegate(x))(\@x2.exists x.(result(x) & publish(@x2,x)))>],
 'a Portuguese delegate published the result': [<ExistsExpression exists x.(result(x) & exists z2.(portugese(z2) & delegate(z2))(\@x2.publish(@x2,x)) & all y.(result(y) -> (y = x)))>,
  <ApplicationExpression exists x.(portugese(x) & delegate(x))(\@x2.exists x.(result(x) & publish(@x2,x) & all y.(result(y) -> (y = x))))>],
 'a Portuguese published a result': [<ExistsExpression exists x.(result(x) & exists z3.(portugese(z3) & publish(z3,x)))>,
  <ExistsExpression exists x.(portugese(x) & exists z4.(result(z4) & publish(x,z4)))>]}

### 1.2 Using Prover9 [4 marks in total]

Show which readings of (1) entail a reading of (2). **[1 mark]**

In [8]:
# There is something missing in this code. You will have to fix it, before you can run it.
#sentences = [
#    'a Portuguese delegate published the result on climate',
#    'a Portuguese delegate published a result',
#    'a Portuguese delegate published the result',
#   'a Portuguese published a result',
#]
from nltk.sem import Expression
prover = nltk.Prover9()
p1 = read_expr('man(socrates)')
p2 = read_expr('all x.(man(x) -> mortal(x))')
c  = read_expr('mortal(socrates)')
print(prover.prove(c, [p1,p2]))

#p1 = read_expr('portuguese(delegate)')
#p2 = read_expr('exists x. (portuguese(x) -> publish(x,y)) x.(man(x) -> mortal(x))')
#c  = read_expr('mortal(socrates)')

# ∃ 𝑥.(𝑥(𝜆 𝑧9.(𝑟𝑒𝑠𝑢𝑙𝑡(𝑧9) ∧ 𝑐𝑙𝑖𝑚𝑎𝑡𝑒(𝑦3) ∧ 𝑜𝑛(𝑧9,𝑦3))) ∧ ∃ 𝑧10.(𝑝𝑜𝑟𝑡𝑢𝑔𝑒𝑠𝑒(𝑧10) ∧ 𝑑𝑒𝑙𝑒𝑔𝑎𝑡𝑒(𝑧10) ∧ 𝑝𝑢𝑏𝑙𝑖𝑠ℎ(𝑧10,𝑥)) ∧ ∀ 𝑦.(𝑦(𝜆 𝑥.(𝑟𝑒𝑠𝑢𝑙𝑡(𝑥) ∧ 𝑐𝑙𝑖𝑚𝑎𝑡𝑒(𝑦3) ∧ 𝑜𝑛(𝑥,𝑦3))) → (𝑥 = 𝑦)))
#e1 = 'exists x.(x(\ y.(result(y)&climate(z)&on(y,z)))&exists v.(portuguese(v)&deledate(v)&publish(v,x))&all w.(w(\ x.(result(x)&climate(z)&on(x,z))) -> (x=w)))'
e1 = 'exists x.((\ y.(result(y)&climate(z)&on(y,z)))&exists v.(portuguese(v)&deledate(v)&publish(v,x))&all w.((\ x.(result(x)&climate(z)&on(x,z))) -> (x=w)))'
p1 = read_expr(e1)
#∃ 𝑥.(𝑟𝑒𝑠𝑢𝑙𝑡(𝑥) ∧ ∃ 𝑧12.(𝑝𝑜𝑟𝑡𝑢𝑔𝑒𝑠𝑒(𝑧12) ∧ 𝑑𝑒𝑙𝑒𝑔𝑎𝑡𝑒(𝑧12) ∧ 𝑝𝑢𝑏𝑙𝑖𝑠ℎ(𝑧12,𝑥)))
e2 = 'exists x.(result(x)&exists y. (portuguese(z)&delegate(y)&publish(y,x)))'
p2 = read_expr(e2)

#∃ 𝑥.(𝑟𝑒𝑠𝑢𝑙𝑡(𝑥) ∧ ∃ 𝑧16.(𝑝𝑜𝑟𝑡𝑢𝑔𝑒𝑠𝑒(𝑧16) ∧ 𝑝𝑢𝑏𝑙𝑖𝑠ℎ(𝑧16,𝑥)))
e3 = 'exists x.(result(x)&exists y. (portuguese(y)&publish(y,x)))'
c = read_expr(e3)
#c  = read_expr('mortal(socrates)')

q = prover.prove(c, [p1,p2])
print(q)

def apply_theorem_prover(premises, goals):
    for premise in premises:
        for goal in goals:
            q = prover.prove(read_expr(premise), read_expr(goal))
            print("Premise      : %s" % (premise))
            print("Goal         : %s" % (goal))
            print("Prover result: %s" % (...))
            print(10*'----')
            
apply_theorem_prover([sentences[0]],[sentences[1]])


True


Prover9FatalException: (FATAL)
%%ERROR: A term cannot be constructed from the marked string:


    exists x ((%%START ERROR%%\y.%%END ERROR%%

Fatal error:  sread_term error

Do all the readings of (1) entail all the readings of (2)? If not, why not? **[1 mark]**

Which readings of (1) entail a reading of (4). **[1 mark]** 

In [9]:
# There is something missing in this code. You will have to fix it, before you can run it.

apply_theorem_prover(...,...)

TypeError: 'ellipsis' object is not iterable

Do all the readings of (1) entail all the readings of (4)? If not, why not? **[1 mark]**


### 1.3 Using Mace [2 marks in total]

Show whether (1) entails (3). **[1 mark]**

In [10]:
# There is something missing in this code. 

def apply_model_builder(premises,goals):
    for premise in premises:
        for goal in goals:
            print("Premise : %s" % (premise))
            print("Goal    : %s" % (goal))
            mc = nltk.MaceCommand(..., assumptions=...)
            ...
            print("a model:")
            print(mc.valuation)
            print(10*'...')

apply_model_builder(...,...)

TypeError: 'ellipsis' object is not iterable

Explain why. **[1 mark]**

## Marks

This part of the assignment has a total of 12 marks.